# Data Pre Processing and Modelling

### Pre-Processing

In [34]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from fastapi import FastAPI, Request
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

In [35]:
data = pd.read_csv('data_wrangled_EDA_logs.csv')

In [36]:
data.head()

,LineId,Date,Time,Level,Component,Content,EventId,EventTemplate,LogType,AdditionalInformation,timestamp,input,tokens,Hour,Token_Count
0,1,2015-10-18,"18:01:47,978",INFO,org.apache.hadoop.mapreduce.v2.app.MRAppMaster,Created MRAppMaster for application appattempt...,E29,created mrappmaster for application appattempt...,hadoop,Process: main,2015-10-18 18:01:47.978,"{""Timestamp"": ""2015-10-18T18:01:47.978000"", ""L...","['created', 'mr', '##app', '##master', 'for', ...",18,12
1,2,2015-10-18,"18:01:48,963",INFO,org.apache.hadoop.mapreduce.v2.app.MRAppMaster,Executing with tokens:,E42,executing with tokens:,hadoop,Process: main,2015-10-18 18:01:48.963,"{""Timestamp"": ""2015-10-18T18:01:48.963000"", ""L...","['executing', 'with', 'token', '##s', ':']",18,5
2,3,2015-10-18,"18:01:48,963",INFO,org.apache.hadoop.mapreduce.v2.app.MRAppMaster,"Kind: YARN_AM_RM_TOKEN, Service: , Ident: (app...",E61,"kind: yarn_am_rm_token, service: , ident: (app...",hadoop,Process: main,2015-10-18 18:01:48.963,"{""Timestamp"": ""2015-10-18T18:01:48.963000"", ""L...","['kind', ':', 'yarn', '_', 'am', '_', 'rm', '_...",18,44
3,4,2015-10-18,"18:01:49,228",INFO,org.apache.hadoop.mapreduce.v2.app.MRAppMaster,Using mapred newApiCommitter.,E111,using mapred newapicommitter.,hadoop,Process: main,2015-10-18 18:01:49.228,"{""Timestamp"": ""2015-10-18T18:01:49.228000"", ""L...","['using', 'map', '##red', 'new', '##ap', '##ic...",18,10
4,5,2015-10-18,"18:01:50,353",INFO,org.apache.hadoop.mapreduce.v2.app.MRAppMaster,OutputCommitter set in config null,E76,outputcommitter set in config null,hadoop,Process: main,2015-10-18 18:01:50.353,"{""Timestamp"": ""2015-10-18T18:01:50.353000"", ""L...","['output', '##com', '##mit', '##ter', 'set', '...",18,10


In [66]:
data['Level'].value_counts()

Level
WARN     2126
INFO     1709
ERROR     163
FATAL       2
Name: count, dtype: int64

In [37]:
warn_data = data[data['Level']=='WARN']
warn_data

,LineId,Date,Time,Level,Component,Content,EventId,EventTemplate,LogType,AdditionalInformation,timestamp,input,tokens,Hour,Token_Count
847,848,2015-10-18,"18:05:27,570",WARN,org.apache.hadoop.ipc.Client,Address change detected. Old: msra-sa-41/10.19...,E10,address change detected. old: <var>/<var>:<var...,hadoop,Process: LeaseRenewer:msrabi@msra-sa-41:9000,2015-10-18 18:05:27.570,"{""Timestamp"": ""2015-10-18T18:05:27.570000"", ""L...","['address', 'change', 'detected', '.', 'old', ...",18,16
848,849,2015-10-18,"18:05:27,570",WARN,org.apache.hadoop.hdfs.LeaseRenewer,Failed to renew lease for [DFSClient_NONMAPRED...,E44,failed to renew lease for [dfsclient_nonmapred...,hadoop,Process: LeaseRenewer:msrabi@msra-sa-41:9000,2015-10-18 18:05:27.570,"{""Timestamp"": ""2015-10-18T18:05:27.570000"", ""L...","['failed', 'to', 'renew', 'lease', 'for', '[',...",18,32
849,850,2015-10-18,"18:05:28,570",WARN,org.apache.hadoop.ipc.Client,Address change detected. Old: msra-sa-41/10.19...,E10,address change detected. old: <var>/<var>:<var...,hadoop,Process: LeaseRenewer:msrabi@msra-sa-41:9000,2015-10-18 18:05:28.570,"{""Timestamp"": ""2015-10-18T18:05:28.570000"", ""L...","['address', 'change', 'detected', '.', 'old', ...",18,16
850,851,2015-10-18,"18:05:28,570",WARN,org.apache.hadoop.hdfs.LeaseRenewer,Failed to renew lease for [DFSClient_NONMAPRED...,E44,failed to renew lease for [dfsclient_nonmapred...,hadoop,Process: LeaseRenewer:msrabi@msra-sa-41:9000,2015-10-18 18:05:28.570,"{""Timestamp"": ""2015-10-18T18:05:28.570000"", ""L...","['failed', 'to', 'renew', 'lease', 'for', '[',...",18,32
851,852,2015-10-18,"18:05:29,570",WARN,org.apache.hadoop.ipc.Client,Address change detected. Old: msra-sa-41/10.19...,E10,address change detected. old: <var>/<var>:<var...,hadoop,Process: LeaseRenewer:msrabi@msra-sa-41:9000,2015-10-18 18:05:29.570,"{""Timestamp"": ""2015-10-18T18:05:29.570000"", ""L...","['address', 'change', 'detected', '.', 'old', ...",18,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3970,1971,2015-07-30,"20:36:32,026",WARN,0.0.0.0/0.0.0.0:2181:ZooKeeperServer,Connection request from old client /10.10.34.1...,E12,connection request from old client /<var>:<var...,zookeeper,Node: NIOServerCxn.Factory Id: 793,2015-07-30 20:36:32.026,"{""Timestamp"": ""2015-07-30T20:36:32.026000"", ""L...","['connection', 'request', 'from', 'old', 'clie...",20,21
3973,1974,2015-07-30,"23:03:42,330",WARN,0.0.0.0/0.0.0.0:2181:NIOServerCnxn,caught end of stream exception,E6,caught end of stream exception,zookeeper,Node: NIOServerCxn.Factory Id: 349,2015-07-30 23:03:42.330,"{""Timestamp"": ""2015-07-30T23:03:42.330000"", ""L...","['caught', 'end', 'of', 'stream', 'exception']",23,5
3981,1982,2015-07-31,"00:16:12,152",WARN,0.0.0.0/0.0.0.0:2181:NIOServerCnxn,caught end of stream exception,E6,caught end of stream exception,zookeeper,Node: NIOServerCxn.Factory Id: 349,2015-07-31 00:16:12.152,"{""Timestamp"": ""2015-07-31T00:16:12.152000"", ""L...","['caught', 'end', 'of', 'stream', 'exception']",0,5
3985,1986,2015-07-31,"11:09:02,264",WARN,0.0.0.0/0.0.0.0:2181:ZooKeeperServer,Connection request from old client /10.10.34.3...,E12,connection request from old client /<var>:<var...,zookeeper,Node: NIOServerCxn.Factory Id: 793,2015-07-31 11:09:02.264,"{""Timestamp"": ""2015-07-31T11:09:02.264000"", ""L...","['connection', 'request', 'from', 'old', 'clie...",11,21


In [64]:
info_data = data[data['Level']=='INFO']
info_data.head()

,LineId,Date,Time,Level,Component,Content,EventId,EventTemplate,LogType,AdditionalInformation,timestamp,input,tokens,Hour,Token_Count,target
0,1,2015-10-18,"18:01:47,978",INFO,org.apache.hadoop.mapreduce.v2.app.MRAppMaster,Created MRAppMaster for application appattempt...,E29,created mrappmaster for application appattempt...,hadoop,Process: main,2015-10-18 18:01:47.978,"{""Timestamp"": ""2015-10-18T18:01:47.978000"", ""L...","['created', 'mr', '##app', '##master', 'for', ...",18,12,2
1,2,2015-10-18,"18:01:48,963",INFO,org.apache.hadoop.mapreduce.v2.app.MRAppMaster,Executing with tokens:,E42,executing with tokens:,hadoop,Process: main,2015-10-18 18:01:48.963,"{""Timestamp"": ""2015-10-18T18:01:48.963000"", ""L...","['executing', 'with', 'token', '##s', ':']",18,5,2
2,3,2015-10-18,"18:01:48,963",INFO,org.apache.hadoop.mapreduce.v2.app.MRAppMaster,"Kind: YARN_AM_RM_TOKEN, Service: , Ident: (app...",E61,"kind: yarn_am_rm_token, service: , ident: (app...",hadoop,Process: main,2015-10-18 18:01:48.963,"{""Timestamp"": ""2015-10-18T18:01:48.963000"", ""L...","['kind', ':', 'yarn', '_', 'am', '_', 'rm', '_...",18,44,2
3,4,2015-10-18,"18:01:49,228",INFO,org.apache.hadoop.mapreduce.v2.app.MRAppMaster,Using mapred newApiCommitter.,E111,using mapred newapicommitter.,hadoop,Process: main,2015-10-18 18:01:49.228,"{""Timestamp"": ""2015-10-18T18:01:49.228000"", ""L...","['using', 'map', '##red', 'new', '##ap', '##ic...",18,10,2
4,5,2015-10-18,"18:01:50,353",INFO,org.apache.hadoop.mapreduce.v2.app.MRAppMaster,OutputCommitter set in config null,E76,outputcommitter set in config null,hadoop,Process: main,2015-10-18 18:01:50.353,"{""Timestamp"": ""2015-10-18T18:01:50.353000"", ""L...","['output', '##com', '##mit', '##ter', 'set', '...",18,10,2


LineId  Date        Time          Level  Component                                                   Content                                                                                                                                                                EventId  EventTemplate                                                                                                                                                              LogType    AdditionalInformation                    timestamp                input                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [38]:
fatal_data = data[data['Level']=='FATAL']
fatal_data

,LineId,Date,Time,Level,Component,Content,EventId,EventTemplate,LogType,AdditionalInformation,timestamp,input,tokens,Hour,Token_Count
1019,1020,2015-10-18,"18:06:26,029",FATAL,org.apache.hadoop.mapred.TaskAttemptListenerImpl,Task: attempt_1445144423722_0020_m_000002_0 - ...,E101,task: attempt_<var> - exited : java.net.norout...,hadoop,Process: IPC Server handler 13 on 62270,2015-10-18 18:06:26.029,"{""Timestamp"": ""2015-10-18T18:06:26.029000"", ""L...","['task', ':', 'attempt', '_', '<var>', '-', 'e...",18,87
1052,1053,2015-10-18,"18:06:28,217",FATAL,org.apache.hadoop.mapred.TaskAttemptListenerImpl,Task: attempt_1445144423722_0020_m_000001_0 - ...,E101,task: attempt_<var> - exited : java.net.norout...,hadoop,Process: IPC Server handler 4 on 62270,2015-10-18 18:06:28.217,"{""Timestamp"": ""2015-10-18T18:06:28.217000"", ""L...","['task', ':', 'attempt', '_', '<var>', '-', 'e...",18,87


In [39]:
data['input'][0]

'{"Timestamp": "2015-10-18T18:01:47.978000", "LogType": "hadoop", "Component": "org.apache.hadoop.mapreduce.v2.app.MRAppMaster", "Content": "Created MRAppMaster for application appattempt_1445144423722_0020_000001", "EventId": "E29", "EventTemplate": "created mrappmaster for application appattempt_<var>"}'

In [40]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
data['target'] = le.fit_transform(data['Level'])

In [41]:
unique_mapping = data[['target', 'Level']].drop_duplicates().reset_index(drop=True)
print(unique_mapping)

   target  Level
0       2   INFO
1       0  ERROR
2       3   WARN
3       1  FATAL


In [42]:
X = data['input']
y = data['target']

In [43]:
print(y)

0       2
1       2
2       2
3       2
4       2
       ..
3995    2
3996    2
3997    2
3998    2
3999    2
Name: target, Length: 4000, dtype: int64


In [44]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42, stratify=y)

In [45]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=3)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [46]:
tokenizer.add_tokens(["<VAR>"])
model.resize_token_embeddings(len(tokenizer))

Embedding(30523, 768, padding_idx=0)

In [47]:
print(y_train)

933     3
3090    2
2724    3
460     2
3698    3
       ..
89      2
3167    3
1970    0
929     2
2803    3
Name: target, Length: 3200, dtype: int64


In [48]:
from datasets import Dataset

train_dataset = Dataset.from_dict({"text": X_train, "label": y_train})
test_dataset = Dataset.from_dict({"text": X_test, "label": y_test})

def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True)

train_tokenized = train_dataset.map(tokenize, batched=True)
test_tokenized = test_dataset.map(tokenize, batched=True)

Map:   0%|          | 0/3200 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

In [49]:
train_dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 3200
})

In [50]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./model_output",
    per_device_train_batch_size=16,
    num_train_epochs=3,
    logging_dir="./logs",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=test_tokenized,
)
trainer.train()

/opt/homebrew/Caskroom/miniconda/base/lib/python3.12/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
500,0.021200


TrainOutput(global_step=600, training_loss=0.019533191720644633, metrics={'train_runtime': 222.3265, 'train_samples_per_second': 43.18, 'train_steps_per_second': 2.699, 'total_flos': 1271709705830400.0, 'train_loss': 0.019533191720644633, 'epoch': 3.0})

In [51]:
trainer.evaluate()

/opt/homebrew/Caskroom/miniconda/base/lib/python3.12/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'eval_loss': 0.009897788986563683,
 'eval_runtime': 4.6888,
 'eval_samples_per_second': 170.618,
 'eval_steps_per_second': 21.327,
 'epoch': 3.0}

In [54]:
# save the model

In [53]:
trainer.save_model("./distilbert-log-classifier")
tokenizer.save_pretrained("./distilbert-log-classifier")

('./distilbert-log-classifier/tokenizer_config.json',
 './distilbert-log-classifier/special_tokens_map.json',
 './distilbert-log-classifier/vocab.txt',
 './distilbert-log-classifier/added_tokens.json',
 './distilbert-log-classifier/tokenizer.json')

In [ ]:
# Load for Inference

In [55]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("./distilbert-log-classifier")
tokenizer = AutoTokenizer.from_pretrained("./distilbert-log-classifier")

In [56]:
# Predict Severity

In [67]:
def predict_severity(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    outputs = model(**inputs)
    pred = outputs.logits.argmax(dim=1).item()
    print("pred : " ,pred)
    severity_label = le.inverse_transform([pred])[0]  # Converts 0 → 'FATAL', etc.
    return severity_label

In [60]:
# Validating / Testing the Classified error type

In [68]:
log_text = "Failed to renew lease for [DFSClient_NONMAPRED..."
print(predict_severity(log_text))  # Output: 'WARN'

pred :  0
ERROR


In [69]:
log_text_component = "Created MRAppMaster for application appattempt_1445144423722_0020_000001"
print(predict_severity(log_text_component)) 

pred :  2
INFO


# CHATBOT

In [17]:
pip install protobuf

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [70]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("./distilbert-log-classifier", local_files_only=True)
model = AutoModelForSequenceClassification.from_pretrained("./distilbert-log-classifier", local_files_only=True)

In [19]:
log_input = "org.apache.hadoop.mapreduce.v2.app.MRAppMaster | Created MRAppMaster for application appattempt_1445144423722_0020_000001 | created mrappmaster for application appattempt_<VAR>"
tokens = tokenizer(log_input, return_tensors="pt", truncation=True, padding=True)
output = model(**tokens)
predicted_class = torch.argmax(output.logits, dim=1).item()
print(predicted_class)  # Should be an integer like 0, 1, or 2

2


target  Level
0       2   INFO
1       0  ERROR
2       3   WARN
3       1  FATAL

In [26]:
label_map = {0: "ERROR", 1: "FATAL", 2: "INFO", 3: "WARN"}
print(label_map[predicted_class])

INFO


In [22]:
pip install gradio

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [27]:
import gradio as gr

def classify_log(component, content, template):
    input_text = f"{component} | {content} | {template}"
    tokens = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True)
    output = model(**tokens)
    prediction = torch.argmax(output.logits, dim=1).item()
    label_map = {0: "ERROR", 1: "FATAL", 2: "INFO", 3: "WARN"}
    return label_map[prediction]

gr.Interface(
    fn=classify_log,
    inputs=["text", "text", "text"],
    outputs="text",
    title="Log Level Classification Chatbot"
).launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


In [33]:
log_input = "org.apache.hadoop.mapreduce.v2.app.MRAppMaster | Created MRAppMaster for application appattempt_1445144423722_0020_000001 | created mrappmaster for application appattempt_<VAR>"
tokens = tokenizer(log_input, return_tensors="pt", truncation=True, padding=True)
output = model(**tokens)
print(output.logits)

tensor([[-2.0562, -2.7824,  3.8771]], grad_fn=<AddmmBackward0>)
